# Cluster product titles into product category

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import Word2Vec
from pyspark.ml.clustering import KMeans

spark = SparkSession.builder.appName('cluster').getOrCreate()

## Read in 12 dataset

In [2]:
schema = StructType([ \
    StructField("marketplace",       StringType(),    True), \
    StructField("customer_id",       StringType(),    True), \
    StructField("review_id",         StringType(),    True), \
    StructField("product_id",        StringType(),    True), \
    StructField("product_parent",    StringType(),    True), \
    StructField("product_title",     StringType(),    True), \
    StructField("product_category",  StringType(),    True), \
    StructField("star_rating",       IntegerType(),   True), \
    StructField("helpful_votes",     IntegerType(),   True), \
    StructField("total_votes",       IntegerType(),   True), \
    StructField("vine",              StringType(),    True), \
    StructField("verified_purchase", StringType(),    True), \
    StructField("review_headline",   StringType(),    True), \
    StructField("review_body",       StringType(),    True), \
    StructField("review_date",       TimestampType(), True), \
  ])

path = ['archive/amazon_reviews_us_Apparel_v1_00.tsv',
        'archive/amazon_reviews_us_Automotive_v1_00.tsv',
        'archive/amazon_reviews_us_Baby_v1_00.tsv',
        'archive/amazon_reviews_us_Beauty_v1_00.tsv',
        'archive/amazon_reviews_us_Books_v1_02.tsv',
        'archive/amazon_reviews_us_Camera_v1_00.tsv',
        'archive/amazon_reviews_us_Electronics_v1_00.tsv',
        'archive/amazon_reviews_us_Furniture_v1_00.tsv',
        'archive/amazon_reviews_us_Sports_v1_00.tsv',
        'archive/amazon_reviews_us_Grocery_v1_00.tsv',
        'archive/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv',
        'archive/amazon_reviews_us_Music_v1_00.tsv']

data = spark.read.csv(path, schema=schema, header=True, sep='\t', mode='DROPMALFORMED')

## Filter out malformated product category

In [3]:
product_category = ['Sports', 'Baby', 'Apparel', 'Grocery', 'Electronics', 'Automotive', 'Books', 'Music', 'Furniture', 'Personal_Care_Appliances', 'Camera', 'Beauty']
data_filter = data.filter(data.product_category.isin(product_category))

data_filter.count()

37172391

## Select only title and category columns

In [4]:
title = data_filter.select('product_title', 'product_category')

In [5]:
title.show()

+--------------------+----------------+
|       product_title|product_category|
+--------------------+----------------+
|Easy Tool Stainle...|         Apparel|
|V28 Women Cowl Ne...|         Apparel|
|James Fiallo Men'...|         Apparel|
|Belfry Gangster 1...|         Apparel|
|JAEDEN Women's Be...|         Apparel|
|Levi's Boys' 514 ...|         Apparel|
|Minimalist Wallet...|         Apparel|
|Harriton Men's Ba...|         Apparel|
|Jockey Women's Un...|         Apparel|
|Alexander Del Ros...|         Apparel|
|Jockey Women's Un...|         Apparel|
|Bali Passion For ...|         Apparel|
|Lilyette Women's ...|         Apparel|
|Vanity Fair Women...|         Apparel|
|Warner's Women's ...|         Apparel|
|Columbia Women's ...|         Apparel|
|Robes King RK Cla...|         Apparel|
|VIV Collection Be...|         Apparel|
|Fruit of the Loom...|         Apparel|
|Jockey Scrubs Wom...|         Apparel|
+--------------------+----------------+
only showing top 20 rows



## Tokenize title column

In [6]:
tokenizer = Tokenizer(inputCol="product_title", outputCol="words")
wordsData = tokenizer.transform(title)
wordsData.show()

+--------------------+----------------+--------------------+
|       product_title|product_category|               words|
+--------------------+----------------+--------------------+
|Easy Tool Stainle...|         Apparel|[easy, tool, stai...|
|V28 Women Cowl Ne...|         Apparel|[v28, women, cowl...|
|James Fiallo Men'...|         Apparel|[james, fiallo, m...|
|Belfry Gangster 1...|         Apparel|[belfry, gangster...|
|JAEDEN Women's Be...|         Apparel|[jaeden, women's,...|
|Levi's Boys' 514 ...|         Apparel|[levi's, boys', 5...|
|Minimalist Wallet...|         Apparel|[minimalist, wall...|
|Harriton Men's Ba...|         Apparel|[harriton, men's,...|
|Jockey Women's Un...|         Apparel|[jockey, women's,...|
|Alexander Del Ros...|         Apparel|[alexander, del, ...|
|Jockey Women's Un...|         Apparel|[jockey, women's,...|
|Bali Passion For ...|         Apparel|[bali, passion, f...|
|Lilyette Women's ...|         Apparel|[lilyette, women'...|
|Vanity Fair Women...|  

## Learn a mapping from words to Vectors.

In [ ]:
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="words", outputCol="features")
model = word2Vec.fit(wordsData)

result = model.transform(wordsData)
#result.show(truncate=False)
# Save this huge object
result.write.save("word2vec.parquet")

In [2]:
# Reload from the disc
result = spark.read.load("word2vec.parquet")

In [3]:
result.show()

+--------------------+----------------+--------------------+--------------------+
|       product_title|product_category|               words|            features|
+--------------------+----------------+--------------------+--------------------+
|DC Sports Muffler...|      Automotive|[dc, sports, muff...|[0.32148125022649...|
|Thrush 17713 Turb...|      Automotive|[thrush, 17713, t...|[0.18454607890453...|
|Max Reflector Jum...|      Automotive|[max, reflector, ...|[-0.0210069790482...|
|40 Inch LED Light...|      Automotive|[40, inch, led, l...|[0.11790505796670...|
|9 MOON Matt Multi...|      Automotive|[9, moon, matt, m...|[0.03611409006749...|
|Rampage Jeep 7689...|      Automotive|[rampage, jeep, 7...|[-0.1092054396867...|
|Ameritree Mazda C...|      Automotive|[ameritree, mazda...|[0.11809038201018...|
|Dorman 425-176 In...|      Automotive|[dorman, 425-176,...|[0.47730810344219...|
|Dupli-Color Ceram...|      Automotive|[dupli-color, cer...|[-0.1952677858727...|
|4-1/2" Dent Pul

In [4]:
result.count()

37172391

## Trains a k-means model

In [3]:
kmeans = KMeans(featuresCol='features', k=12)
model = kmeans.fit(result)

## Evaluate clustering by computing Within Set Sum of Squared Errors

In [4]:
wssse = model.summary.trainingCost
print(wssse)
print("Within Set Sum of Squared Errors = " + str(wssse))

4512713.083075685
Within Set Sum of Squared Errors = 4512713.083075685


## Shows the center of each cluster

In [5]:
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[-0.06756582  0.06419763 -0.24397379]
[-0.38753292  0.10782893  0.28906787]
[-0.28179655 -0.10252775 -0.6694482 ]
[-0.45903388  0.713647    0.27148985]
[0.89117916 0.12049009 0.30267616]
[0.34814195 0.06660468 0.08241818]
[-0.34397886  0.52769217 -0.19828648]
[-0.22615748  0.62228281 -0.59484177]
[ 0.48361393 -0.01003218 -0.56472573]
[-0.04793732 -0.4490937  -0.36508855]
[-0.38537371 -0.14404441 -0.20167984]
[ 0.3654186  -0.74665895 -0.0526108 ]


## Get the clustering prediction

In [6]:
pred = model.transform(result).select('prediction')
pred.show()

+----------+
|prediction|
+----------+
|        11|
|        11|
|         9|
|         5|
|         9|
|         9|
|         9|
|        11|
|         1|
|         0|
|         0|
|         9|
|        11|
|        11|
|         9|
|         9|
|         0|
|         0|
|        11|
|         9|
+----------+
only showing top 20 rows



In [7]:
pred.count()

37172391

## Calculate the clustering error within each group

In [8]:
df1 = result.withColumn("id", monotonically_increasing_id() )
df2 = pred.withColumn("id", monotonically_increasing_id() )

df = df1.join(df2, df1.id == df2.id, how='inner')
df.show()

+--------------------+----------------+--------------------+--------------------+----+----------+----+
|       product_title|product_category|               words|            features|  id|prediction|  id|
+--------------------+----------------+--------------------+--------------------+----+----------+----+
|YTX14-BS GEL 12V ...|      Automotive|[ytx14-bs, gel, 1...|[0.16303894590390...|  26|         5|  26|
|OPT7 Show Glow CR...|      Automotive|[opt7, show, glow...|[0.34752524892489...|  29|         5|  29|
|6 New NGK Laser P...|      Automotive|[6, new, ngk, las...|[0.16509262192994...| 474|        11| 474|
|Kayo Maxtar Porta...|      Automotive|[kayo, maxtar, po...|[0.21503228770702...| 964|         5| 964|
|Oro-Tek OSC-0087 ...|      Automotive|[oro-tek, osc-008...|[0.42032591719180...|1677|        11|1677|
|Thetford 31705  A...|      Automotive|[thetford, 31705,...|[0.31676158971256...|1697|        11|1697|
|Leading Edge Ligh...|      Automotive|[leading, edge, l...|[0.0722927703

In [ ]:
df.write.save("clustering.parquet")